## Indicating how top 13F filers have performed over time

#### ALL in vs Cash Weighted (Sold out of position(s))

In [55]:
import pandas as pd
import requests
import yfinance as yf
import os
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [56]:
mapping1 = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Final_cusip_map\CUSIP_Mapping.csv", usecols = ["CUSIP", "SYMBOL"])
mapping2 = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Final_cusip_map\CUSIP_Mapping-Brocks-Laptop.csv", usecols = ["CUSIP", "SYMBOL"])
mapping = pd.concat([mapping1, mapping2]).drop_duplicates()


In [58]:
today = "10022024"
historical_data = pd.read_csv(fr"Historical_Price_Data\Stock_Data.csv", parse_dates=['Date'])
historical_dividends = pd.read_csv(fr"Historical_Price_Data\Dividend_data.csv")

In [60]:
holdings = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Holdings_shares\Stanley_Druckenmiller.csv")

holdings = holdings.merge(mapping, on = "CUSIP", how='left')
holdings = holdings[holdings.columns[1:]]

In [64]:
# Define start and end dates
start_date = "2013-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')




def quarterly_alpha(test):
    if 'Unnamed: 0' in test.columns:
        test = test.drop(columns=['Unnamed: 0'])

    # Drop columns that end with '_x' or '_y'
    test = test.drop(columns=[col for col in test.columns if col.endswith('_x') or col.endswith('_y')])

    
    test1 = test.melt(id_vars=['NAME OF ISSUER', 'CUSIP', 'CALL', 'SYMBOL'], var_name='Date', value_name='Shares').drop_duplicates()
    test1["Date"] = pd.to_datetime(test1["Date"])

    ### adding 
    # Get unique dates from historical_data
    historical_dates = historical_data['Date'].drop_duplicates().sort_values()
    q_dates = test1[["Date", "CALL"]].drop_duplicates()

    # Define a function to find the closest date after a given date
    def find_closest_after(date, dates):
        return dates[dates > date].min()
    
    # Apply the function to each row in test1
    q_dates['NextDay'] = q_dates['Date'].apply(lambda x: find_closest_after(x, historical_dates))


    ## getting date of next market open
    test1 = test1.merge(q_dates[["Date", "NextDay"]], on = 'Date', how='left')

    test1.drop(columns="Date",inplace=True)


    test1 = test1.merge(historical_data[["SYMBOL","Date", "Open"]], left_on= ["SYMBOL", "NextDay"], right_on= ["SYMBOL", "Date"], how = 'left').drop_duplicates()
    test1["Starting Value"] = (test1["Open"] * test1["Shares"]).fillna(0)

    test1 = test1[(test1["Starting Value"] > 0) & (test1["CALL"].isna())]

    test1["Quarter"] = np.where(test1["Date"].dt.month <= 3, "Q1",
                            np.where(test1["Date"].dt.month <= 6, "Q2",
                                    np.where(test1["Date"].dt.month <= 9, "Q3","Q4")))

    test1["Year"] = test1["Date"].dt.year
    test1.drop(columns="Date",inplace=True)

    test1["Year_Quarter"] = test1["Year"].astype(str) + "-" + test1["Quarter"] # np.where(test1["Quarter"] == "Q4", (test1["Year"]-1).astype(str) + "-" + test1["Quarter"], test1["Year"].astype(str) + "-" + test1["Quarter"])

    ## joining Quarter end date 
    date_table = test1["NextDay"].drop_duplicates().reset_index()
    date_table = date_table.sort_values('NextDay', ascending = False)
    date_table["QuarterEnd"] = np.where(date_table['NextDay'].shift(1).isna(), historical_dates[len(historical_dates)-1].strftime('%Y-%m-%d'), date_table['NextDay'].shift(1).dt.strftime('%Y-%m-%d'))
    date_table["QuarterEnd"]  = pd.to_datetime(date_table["QuarterEnd"])


    test1 = test1.merge(date_table[["NextDay", "QuarterEnd"]], how = 'left', on = 'NextDay')

    ## Next Quarter opening price (Theoretical selling price)
    historical_data2 = historical_data.rename(columns={"Open": "Open_Next_Quarter"})
    test1 = test1.merge(historical_data2[["SYMBOL","Date", "Open_Next_Quarter"]], left_on= ["SYMBOL", "QuarterEnd"], right_on= ["SYMBOL", "Date"], how = 'left').drop_duplicates()
    test1["Ending Value"] = (test1["Open_Next_Quarter"] * test1["Shares"]).fillna(0)

    spy_index = yf.download('SPY', start=start_date, end=end_date).reset_index()

    returns = test1.groupby(["Year_Quarter", "NextDay", "QuarterEnd"])[["Starting Value", "Ending Value"]].sum().reset_index()
    spy = spy_index[["Date", "Open"]]
    spy.columns = ["Date", "spy_start"]
    spy["spy_end"] = spy["spy_start"]

    returns = returns.merge(spy[["Date", "spy_start"]], left_on = 'NextDay', right_on='Date', how = 'left')
    returns.drop(columns = 'Date', inplace = True)
    returns = returns.merge(spy[["Date", "spy_end"]], left_on = 'QuarterEnd', right_on='Date', how = 'left')
    returns.drop(columns = 'Date', inplace = True)

    returns["Quarterly Return"] = (returns["Ending Value"] - returns["Starting Value"])/returns["Starting Value"]
    returns["Benchmark Return"] = (returns["spy_end"] - returns["spy_start"])/returns["spy_start"]


    returns["Return Ann Roll4"] = (returns["Quarterly Return"] + 1).rolling(window=4).apply(lambda x: x.prod(), raw=True)
    returns["Benchmark Return Ann Roll4"] = (returns["Benchmark Return"] + 1).rolling(window=4).apply(lambda x: x.prod(), raw=True)

    returns["Total Return"] = (returns["Quarterly Return"] + 1).cumprod()
    returns["Benchmark Total Return"] = (returns["Benchmark Return"] + 1).cumprod()

    returns["Alpha"] = returns["Return Ann Roll4"] - returns["Benchmark Return Ann Roll4"]
    
    return returns[["Year_Quarter", "Alpha"]]

In [65]:
holdings.columns

Index(['NAME OF ISSUER', 'CUSIP', 'CALL', '2024-11-14', '2024-08-14',
       '2024-05-15', '2024-02-14', '2023-11-14', '2023-08-14', '2023-05-15',
       '2023-02-14', '2022-11-14', '2022-08-15', '2022-05-16', '2022-02-14',
       '2021-11-15', '2021-08-16', '2021-05-17', '2021-02-16', '2020-11-16',
       '2020-08-14', '2020-05-15', '2020-02-14', '2019-11-14', '2019-08-14',
       '2019-05-15', '2019-02-14', '2018-11-14', '2018-08-14', '2018-05-15',
       '2018-02-14', '2017-11-14', '2017-08-14', '2017-05-15', '2017-02-14',
       '2016-11-14', '2016-08-15', '2016-05-16', '2016-02-16', '2015-11-13',
       '2015-08-14', '2015-05-15', '2015-02-12', '2014-11-14', '2014-08-14',
       '2014-05-14', '2014-02-14', '2013-11-14', '2013-08-14', 'SYMBOL'],
      dtype='object')

In [66]:
test = holdings.copy()

test1 = test.melt(id_vars=['NAME OF ISSUER', 'CUSIP', 'CALL', 'SYMBOL'], var_name='Date', value_name='Shares').drop_duplicates()
test1["Date"] = pd.to_datetime(test1["Date"])

### adding 
# Get unique dates from historical_data
historical_dates = historical_data['Date'].drop_duplicates().sort_values()
q_dates = test1[["Date", "CALL"]].drop_duplicates()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13443234 entries, 0 to 13443233
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   SYMBOL      object 
 2   Date        object 
 3   Open        float64
dtypes: float64(1), int64(1), object(2)
memory usage: 410.3+ MB


In [67]:
directory = "Holdings_shares"

# Create an empty dictionary to store DataFrames
dfs = []

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df_name = os.path.splitext(filename)[0]  # Get the file name without the extension
        
        read_df = pd.read_csv(filepath)
        
        read_df = read_df.merge(mapping[["CUSIP", "SYMBOL"]].drop_duplicates(), how = 'left', on = "CUSIP")

        ## Call Alpha Function
        df = quarterly_alpha(read_df)

        df.columns = ['Year_Quarter', df_name + '_Alpha']

        dfs.append(df)

        print(df_name)

        

# Merge all the DataFrames on 'Year_Quarter' using reduce (ensures no duplicates)
from functools import reduce

final_df = reduce(lambda left, right: pd.merge(left, right, on='Year_Quarter', how='outer'), dfs)

# Display or save the final DataFrame
print(final_df)



[*********************100%%**********************]  1 of 1 completed
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\2986347522.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy["spy_end"] = spy["spy_start"]


ALTAROCK PARTNERS LP


[*********************100%%**********************]  1 of 1 completed
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\2986347522.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy["spy_end"] = spy["spy_start"]


Altimeter


[*********************100%%**********************]  1 of 1 completed
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\2986347522.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy["spy_end"] = spy["spy_start"]


...

VIKING GLOBAL INVESTORS LP


[*********************100%%**********************]  1 of 1 completed
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\2986347522.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy["spy_end"] = spy["spy_start"]


Warren_Buffet


[*********************100%%**********************]  1 of 1 completed
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\2986347522.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy["spy_end"] = spy["spy_start"]


Whale Rock Capital Management LLC
   Year_Quarter  ALTAROCK PARTNERS LP_Alpha  Altimeter_Alpha  Ancora_Alpha  \
0       2015-Q1                         NaN         0.224103     -0.283798   
1       2015-Q2                         NaN         0.258973           NaN   
2       2015-Q3                         NaN         0.115012     -0.503849   
3       2015-Q4                   -0.016722        -0.028423     -0.592418   
4       2016-Q1                    0.026682        -0.088317     -0.673284   
5       2016-Q2                    0.012196        -0.137215     -0.604557   
6       2016-Q3                   -0.010271         0.010351     -0.508324   
7       2016-Q4                    0.146523         0.157254     -0.277395   
8       2017-Q1                   -0.026056         0.313488     -0.225387   
9       2017-Q2                    0.044997         0.201369     -0.236846   
10      2017-Q3                    0.029076        -0.193520     -0.148120   
11      2017-Q4               

In [74]:
final_df2 = final_df.sort_values(by = "Year_Quarter", ascending=False).reset_index(drop=True)

In [81]:
Last_2y_alpha = final_df2.iloc[:8].mean().reset_index()
Last_4y_alpha = final_df2.iloc[:16].mean().reset_index()
Last_2y_std = final_df2.iloc[:8].std().reset_index()
Last_4y_std = final_df2.iloc[:16].std().reset_index()

whale_alphas = Last_2y_alpha.merge(Last_4y_alpha, on = "index")
whale_alphas = whale_alphas.merge(Last_2y_std, on = "index")
whale_alphas = whale_alphas.merge(Last_4y_std, on = "index")

whale_alphas.columns = ["Whale", "2Y_Alph", "4Y_Alph", "2Y_std", "4Y_std"]

whale_alphas.sort_values(by = "2Y_Alph", ascending=False)

C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\1868465420.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Last_2y_alpha = final_df2.iloc[:8].mean().reset_index()
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\1868465420.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Last_4y_alpha = final_df2.iloc[:16].mean().reset_index()
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\1868465420.py:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In ad

,Whale,2Y_Alph,4Y_Alph,2Y_std,4Y_std
25,"Valley Forge Capital Management, LP_Alpha",0.189574,0.059990,0.045443,0.138079
22,Stanley_Druckenmiller_Alpha,0.170464,0.047432,0.105660,0.159769
24,TIGER GLOBAL MANAGEMENT LLC_Alpha,0.155189,-0.098304,0.139222,0.299058
0,ALTAROCK PARTNERS LP_Alpha,0.143796,0.019083,0.114168,0.151201
8,David_Tepper_Alpha,0.098714,0.013194,0.126511,0.151908
5,"Brave Warrior Advisors, LLC_Alpha",0.094201,0.158258,0.080795,0.114543
1,Altimeter_Alpha,0.065285,-0.093557,0.158924,0.242107
16,LONE PINE CAPITAL LLC_Alpha,0.057359,-0.085620,0.120721,0.202756
13,Himalaya Capital Management LLC_Alpha,0.037135,0.012156,0.067300,0.118558
28,Whale Rock Capital Management LLC_Alpha,0.035431,-0.155391,0.335875,0.336167


In [78]:
final_df2

,Year_Quarter,ALTAROCK PARTNERS LP_Alpha,Altimeter_Alpha,Ancora_Alpha,"ATLANTIC INVESTMENT MANAGEMENT, INC._Alpha",Bill_Ackman_Alpha,"Brave Warrior Advisors, LLC_Alpha",COATUE MANAGEMENT LLC_Alpha,Daniel_Loeb_Alpha,David_Tepper_Alpha,...,Nelson_Peltz_Alpha,Punch Card Management L.P._Alpha,SEMPER AUGUSTUS INVESTMENTS GROUP LLC_Alpha,Stanley_Druckenmiller_Alpha,"Stokes Family Office, LLC_Alpha",TIGER GLOBAL MANAGEMENT LLC_Alpha,"Valley Forge Capital Management, LP_Alpha",VIKING GLOBAL INVESTORS LP_Alpha,Warren_Buffet_Alpha,Whale Rock Capital Management LLC_Alpha
0,2024-Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.061973,NaN,NaN,NaN,NaN,NaN
1,2024-Q3,-0.025004,-0.116292,-0.149329,-0.218842,-0.088051,0.043260,0.151059,0.044011,-0.023149,...,-0.068461,-0.048507,-0.131586,0.359214,-0.101397,0.193415,0.250750,-0.314136,-0.070025,0.198140
2,2024-Q2,0.054389,-0.046045,-0.175172,-0.173384,-0.129709,0.020886,0.036827,-0.009549,0.015090,...,-0.148850,-0.013835,-0.102055,0.144619,-0.093299,0.121237,0.239923,-0.368327,-0.028203,0.064958
3,2024-Q1,0.143022,-0.003828,-0.181734,0.064963,-0.110522,0.222469,0.210338,0.030299,0.223596,...,-0.092655,0.116408,-0.097264,0.262886,-0.108898,0.294483,0.143388,-0.143297,-0.131851,0.242660
4,2023-Q4,0.258126,0.343875,-0.198876,-0.157928,-0.052963,0.087661,0.094578,-0.065947,0.286330,...,-0.115251,-0.113160,-0.280662,0.125983,-0.091988,0.300475,0.203746,-0.015895,-0.128961,0.334020
5,2023-Q3,0.308839,0.107146,-0.167581,-0.040128,-0.027653,0.072829,-0.028473,-0.114497,0.154879,...,-0.123194,-0.143043,-0.248717,0.040488,-0.065301,0.226330,0.169679,-0.146849,-0.075159,0.289033
6,2023-Q2,0.109113,-0.016814,-0.115367,0.056817,0.063236,0.189695,-0.214881,-0.071875,0.075959,...,-0.019967,-0.203102,-0.098260,0.128500,-0.028547,0.015890,0.131431,-0.116309,-0.050670,-0.452391
7,2023-Q1,0.158087,0.188956,-0.116953,-0.013738,0.137207,0.022606,-0.188923,0.017375,-0.041712,...,0.021021,-0.240746,-0.142129,0.131558,-0.008349,-0.065507,0.188101,-0.038021,-0.005772,-0.428402
8,2022-Q4,-0.038336,-0.285673,-0.071068,0.068079,0.065732,0.277672,-0.323318,-0.070858,0.084362,...,0.004197,-0.171929,0.087363,-0.059899,0.009102,-0.312061,0.059555,-0.331424,-0.041396,-0.549884
9,2022-Q3,-0.144653,-0.443597,-0.058016,0.036271,0.068483,0.331976,-0.466972,-0.188509,0.092853,...,0.001342,-0.187073,0.097752,-0.083252,0.023757,-0.486683,0.029150,-0.557197,0.078649,-0.624852


In [43]:
quarterly_alpha(holdings)

[*********************100%%**********************]  1 of 1 completed
C:\Users\bfass\AppData\Local\Temp\ipykernel_5380\2627400107.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy["spy_end"] = spy["spy_start"]


,Year_Quarter,Alpha
0,2013-Q3,NaN
1,2013-Q4,NaN
2,2014-Q1,NaN
3,2014-Q2,0.329624
4,2014-Q3,0.143293
5,2014-Q4,0.029621
6,2015-Q1,0.011611
7,2015-Q2,-0.099191
8,2015-Q3,-0.019781
9,2015-Q4,-0.028093


In [ ]:

# Get unique dates from historical_data
historical_dates = historical_data['Date'].drop_duplicates().sort_values()
q_dates = test1[["NextDay", "Quarter"]].drop_duplicates()

# Define a function to find the closest date after a given date
def find_closest_after(date, dates):
    return dates[dates > date].min()

# Apply the function to each row in test1
q_dates['Closest_Historical_Date'] = q_dates['NextDay'].apply(lambda x: find_closest_after(x, historical_dates))

In [ ]:
q_dates

,NextDay,Quarter,Closest_Historical_Date
0,2024-08-15,Q3,2024-08-16
109,2024-05-16,Q2,2024-05-17
228,2024-02-15,Q1,2024-02-16
324,2023-11-15,Q4,2023-11-16
413,2023-08-15,Q3,2023-08-16
511,2023-05-16,Q2,2023-05-17
604,2023-02-15,Q1,2023-02-16
692,2022-11-15,Q4,2022-11-16
796,2022-08-16,Q3,2022-08-17
860,2022-05-17,Q2,2022-05-18
